In [2]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT  
   MIN(DATE(invoice_date)) AS first_order,
   MAX(DATE(invoice_date)) AS last_order
  FROM invoice;

Done.


first_order,last_order
2017-01-03,2020-12-30


In [8]:
%%sql
SELECT billing_country AS country_name,
 COUNT(DISTINCT customer_id) AS customer_count
  FROM invoice
 GROUP BY 1
 ORDER BY 2 DESC;   

Done.


country_name,customer_count
USA,13
Canada,8
Brazil,5
France,5
Germany,4
United Kingdom,3
Czech Republic,2
India,2
Portugal,2
Argentina,1


In [3]:
%%sql
WITH 
    genre_usa AS
            (SELECT 
                g.name genre,
                g.genre_id genre_id
            FROM genre g 
            INNER JOIN track t ON t.genre_id = g.genre_id
            INNER JOIN invoice_line il ON t.track_id = il.track_id
            INNER JOIN invoice i ON il.invoice_id = i.invoice_id
            WHERE i.billing_country = 'USA'
            )
SELECT 
      genre,
 COUNT(genre_id) AS number_of_tracks,
 ROUND(CAST(COUNT(genre_id) AS FLOAT)/(SELECT COUNT(*) FROM genre_usa) * 100, 2) AS percentage
  FROM genre_usa
 GROUP BY genre
 ORDER BY number_of_tracks DESC;   

Done.


genre,number_of_tracks,percentage
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


In [4]:
%%sql
SELECT
    e.first_name || ' ' || e.last_name AS sales_rep,
    e.birthdate AS birth_date,
    e.hire_date AS hire_date,
    e.city AS work_location,
    ROUND(SUM(i.total), 2) AS total_sales
  FROM invoice i
 INNER JOIN customer c ON c.customer_id = i.customer_id
 INNER JOIN employee e ON e.employee_id = c.support_rep_id
 GROUP BY sales_rep
 ORDER BY total_sales DESC;

Done.


sales_rep,birth_date,hire_date,work_location,total_sales
Jane Peacock,1973-08-29 00:00:00,2017-04-01 00:00:00,Calgary,1731.51
Margaret Park,1947-09-19 00:00:00,2017-05-03 00:00:00,Calgary,1584.0
Steve Johnson,1965-03-03 00:00:00,2017-10-17 00:00:00,Calgary,1393.92


In [5]:
%%sql
WITH 

    country_customers AS 
    (
        SELECT 
            country,
            COUNT(*) AS customers,
            CASE
                WHEN COUNT(*) = 1 THEN 'Other'
                ELSE country
            END AS countries
          FROM customer
         GROUP BY country
    ),
    
    totals AS
    (
        SELECT 
            c.country,
            ROUND(SUM(i.total), 2) total_value,
            COUNT(*) total_sales,
            ROUND(SUM(i.total) / COUNT(*), 2) AS avg_value
        FROM invoice i
        INNER JOIN customer c ON c.customer_id = i.customer_id
        GROUP BY c.country
        ORDER BY total_value DESC
    )

SELECT 
    cc.countries, 
    sum(cc.customers) AS customers, 
    t.total_sales,
    t.total_value, 
    t.avg_value,
    ROUND(CAST(t.total_sales AS Float) / sum(cc.customers), 2) AS avg_customer_sales
  FROM country_customers cc
 INNER JOIN totals t ON t.country = cc.country
 GROUP BY countries
 ORDER BY total_value DESC;

Done.


countries,customers,total_sales,total_value,avg_value,avg_customer_sales
USA,13,131,1040.49,7.94,10.08
Canada,8,76,535.59,7.05,9.5
Brazil,5,61,427.68,7.01,12.2
France,5,50,389.07,7.78,10.0
Germany,4,41,334.62,8.16,10.25
Czech Republic,2,30,273.24,9.11,15.0
United Kingdom,3,28,245.52,8.77,9.33
Portugal,2,29,185.13,6.38,14.5
India,2,21,183.15,8.72,10.5
Other,15,10,75.24,7.52,0.67


In [6]:
%%sql
WITH 
invoices_first_track AS 
(
    SELECT invoice_id, MIN(track_id) AS first_track_id
      FROM invoice_line
  GROUP BY invoice_id
),
invoices_albums AS
(
    SELECT ift.*,
           CASE
           WHEN
                (
                  SELECT il.track_id 
                    FROM invoice_line AS il 
                   WHERE il.invoice_id = ift.invoice_id
                    
                  EXCEPT
                    
                  SELECT track_id 
                    FROM track 
                   WHERE album_id =
                      (
                        SELECT album_id 
                          FROM track 
                         WHERE track_id = ift.first_track_id
                      )
                ) IS NULL
    
                AND 
    
                (
                  SELECT track_id 
                    FROM track 
                   WHERE album_id =
                    (
                        SELECT album_id 
                          FROM track 
                         WHERE track_id = ift.first_track_id
                    )
                    
                 EXCEPT
                    
                 SELECT il.track_id 
                   FROM invoice_line AS il 
                  WHERE il.invoice_id = ift.invoice_id
               ) IS NULL THEN 'YES'
           ELSE 'NO'
           END AS album_purchase
    FROM invoices_first_track AS ift
)
  SELECT
          album_purchase,
          COUNT(invoice_id) AS number_of_invoices,
          ROUND(CAST(COUNT(invoice_id) AS FLOAT) / 
                        (SELECT COUNT(*) FROM invoice),2) * 100 AS percent_of_invoices
    FROM invoices_albums 
GROUP BY album_purchase;

Done.


album_purchase,number_of_invoices,percent_of_invoices
NO,500,81.0
YES,114,19.0
